In [1]:
from imports import *
from functions import *
from auth import *
from constants import *
# Active l'extension autoreload dans un notebook Jupyter
%load_ext autoreload
%autoreload 2

In [2]:
driver = create_driver()

In [3]:
wait = WebDriverWait(driver, timeout=3)

In [4]:
driver.get(website_url)
mi_cuenta = wait.until(EC.presence_of_element_located((By.XPATH, "//span[normalize-space()='Mi cuenta']")))  # Remplacer par l'élément attendu
mi_cuenta.click()

In [ ]:
# Envoie de l'email sur le site
try:
    ingresar_email_zone = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Ingresa tu email']")))
    ingresar_email_zone.clear()
    ingresar_email_zone.send_keys(EMAIL)
    ingresar_email_zone.send_keys(Keys.ENTER)
except:
    print('NO')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=786169607261-vlr2d3cvfagt9q8ngt7n1ub4tcececdi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54444%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=IXiNA6hIPs29hnJQXm6uu0ny5YIfW4&access_type=offline


In [ ]:
try:
    ingresar_email_zone = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Ingresa tu email']")))
    ingresar_email_zone.clear()
    ingresar_email_zone.send_keys(email)
    ingresar_email_zone.send_keys(Keys.ENTER)
except:
    print('NO')

In [ ]:
main_page=driver.current_url
hamburger_button = wait.until(EC.presence_of_element_located((By.XPATH, "//*[name()='path' and @id='Trazado_7945']")))
hamburger_button.click()

Go on the Computer Page

In [ ]:
category = "Tecnología"
article_type_choice = "Portátiles"

main_page=driver.current_url
def go_to_article_page(driver, category, article_type):
    import time
    from selenium.webdriver.common.by import By

    try:
        # Récupère le menu des catégories
        categories_menu = driver.find_element(By.XPATH, '//*[@id="header-page"]/aside/ul/section[2]')

        # Cherche tous les div ayant l'attribut data-link-content="true"
        divs_with_data_link_content = categories_menu.find_elements(By.XPATH, './/div[@data-link-content="true"]')

        # Clique sur la catégorie principale
        for div in divs_with_data_link_content:
            if div.text.strip() == category:
                div.click()
                print(f"Catégorie '{category}' cliquée.")
                time.sleep(1)
                break

        # Menu déroulé : chercher le type d'article
        selected_menu = driver.find_element(By.XPATH, './/ul[@data-content-list="true"]')
        list_elements = selected_menu.find_elements(By.TAG_NAME, 'li')

        for li in list_elements:
            if li.text.strip() == article_type:
                try:
                    link = li.find_element(By.TAG_NAME, 'a')
                    link.click()
                    print(f"Lien cliqué vers la page '{article_type}'")
                    time.sleep(2)
                    return driver.current_url
                except:
                    print(f"Lien introuvable dans l'élément '{article_type}'")
                    return None
        print(f"Type d'article '{article_type}' non trouvé.")
        return None

    except Exception as e:
        print(f"Erreur lors de la navigation : {e}")
        return None
try:
    go_to_article_page(driver, category, article_type_choice)
except Exception as e:
    print(str(e))

In [ ]:
# Récupère le menu contenant tous les filtres (accordéons)
computer_filter_menu = driver.find_element(By.XPATH, "//div[@class='accordion_fs-accordion__mi4MR']")

# Récupère tous les titres des filtres (balises h4)
filters = computer_filter_menu.find_elements(By.TAG_NAME, 'h4')

# Dictionnaire pour stocker les filtres trouvés
filters_dict = {}

# Parcours et affiche les filtres
for f in filters:
    filter_name = f.text.strip()
    print(filter_name)
    filters_dict[filter_name] = f  # Stocke le WebElement par nom

# Exemple : accéder à un filtre spécifique
if "Categoria" in filters_dict:
    categoria_filter = filters_dict["Categoria"]
    categoria_filter.click()  # Déroule le filtre si besoin
    print("Filtre 'Categoria' sélectionné.")

import json

# Crée un dictionnaire avec juste les noms des filtres (clés) sans les WebElement
filters_names = list(filters_dict.keys())

# Sauvegarde dans un fichier JSON ou affiche
with open("computers_filters.json", "w", encoding="utf-8") as f:
    json.dump(filters_names, f, ensure_ascii=False, indent=2)

# Ou juste afficher proprement :
print(json.dumps(filters_names, indent=2, ensure_ascii=False))


Analyser Catégorie

In [ ]:
Categoría = next((f for f in filters if f.text == "Categoría"), None)
# Trouver toutes les checkboxes (éléments <input type="checkbox">) à l'intérieur de l'élément 'Categoría'
checkboxes = Categoría.find_elements(By.XPATH, './/input[@type="checkbox"]')
checkboxes

In [ ]:
def click_with_JS(element, driver):
    driver.execute_script("arguments[0].click();", element)

In [ ]:
try:
    checkbox_computador = driver.find_element(By.XPATH, '//*[@id="desktop-store-filter-Categoría-Computadores"]')
    click_with_JS(checkbox_computador, driver)  
    print(get_all_attributes(checkbox_computador, driver))
except Exception as e:
    print(str(e))

Analysing Sub Categoria

In [ ]:
Sub_categoría = filters[1]

# desktop-store-filter-Sub-categoría-Accesorios de cámaras

# Vérifier si l'élément Sub_categoría a été trouvé
if Sub_categoría:
    # Trouver toutes les checkboxes à l'intérieur de 'Sub_categoría'
    checkboxes = Sub_categoría.find_elements(By.XPATH, '//input[@type="checkbox" and @data-fs-checkbox="true"]')
    print(f"Nombre de checkboxes trouvées : {len(checkboxes)}")
    for checkbox in checkboxes:
        print(checkbox.get_attribute("data-value"))
else:
    print("L'élément 'Sub-categoría' n'a pas été trouvé.")

In [ ]:
one_checkbox =  checkboxes[0]
get_all_attributes(one_checkbox, driver)

In [ ]:
# Liste des éléments que l'utilisateur veut
filters_choice = [
    "asus", 
    "accesorios de camaras", 
    "televisores samsung", 
    "acer", 
    "smartwatch",
]

# Fonction pour cocher les checkboxes en fonction des éléments voulus
from unidecode import unidecode

def cocher_checkboxes(filters_choice, checkboxes):
    for checkbox in checkboxes:
        # Récupère le data-value de la checkbox, remplace les "-" par des espaces et enlève les accents
        data_value = unidecode(checkbox.get_attribute("data-value").replace("-", " ").lower())
        
        # Vérifie si l'un des éléments voulus est contenu dans le data-value
        for item in filters_choice:
            # Enlève aussi les accents de l'élément de la liste de filtres
            item_normalized = unidecode(item.lower())
            if item_normalized in data_value:
                checkbox.click()  # Coche la checkbox si l'élément est trouvé
                print(f"Checkbox pour '{data_value}' cochée.")
                break  # Sort de la boucle dès qu'une correspondance est trouvée
